In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
import rdflib
import pandas as pd
from fastai.tabular import *
import sys
sys.path.append("../src")

In [6]:
from data_loader import from_zipfile, SUBJ_RT

In [4]:
prod_graphs = from_zipfile("/Users/jpnelson/Downloads/sinopia_export_all_2020-05-04T22_03_43.280Z.zip")

Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/ucdavis_2020-05-04T22:03:44.322Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/northwestern_2020-05-04T22:03:55.130Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/cornell_2020-05-04T22:03:43.320Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/minnesota_2020-05-04T22:03:55.371Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/boulder_2020-05-04T22:03:57.903Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/chicago_2020-05-04T22:03:44.206Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/yale_2020-05-04T22:03:58.344Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/nlm_2020-05-04T22:03:53.737Z/complete.log
Failed to parse sinopia_export_all_2020-05-04T22:03:43.280Z/princeton_2020-05-04T22:03:43.517Z/complete.log
Failed to parse sinopia_export_all_2020-05-0

In [5]:
len(prod_graphs)

2807

In [127]:
def graph_triples_df(graph: rdflib.Graph) -> pd.DataFrame:
    data = [(str(s), str(p), str(o)) for s,p,o in graph.skolemize(authority=f"https://{graph.identifier}.sinopia.io//")]
    result = graph.query(SUBJ_RT)
    if len(result.bindings) < 1:
        return # Doesn't try to return anything
    resource_template = str(graph.query(SUBJ_RT).bindings[0]['resource_template'])
    df = pd.DataFrame(data=data, columns=['Subject', 'Predicate', 'Object'])
    df['resource_template'] = resource_template
    return df

In [121]:
df0 = graph_triples_df(prod_graphs[0])

In [131]:
frames = []
for i,graph in enumerate(prod_graphs):
    df = graph_triples_df(graph)
    if df is None:
        print(f"Failed to add {i}")
    else:
        frames.append(df)

Failed to add 1620
Failed to add 1621
Failed to add 1622
Failed to add 1623
Failed to add 1624
Failed to add 1625
Failed to add 1626
Failed to add 1627
Failed to add 1628
Failed to add 1629
Failed to add 1630
Failed to add 1631
Failed to add 1632
Failed to add 1633
Failed to add 1634
Failed to add 1635
Failed to add 1636
Failed to add 1637
Failed to add 1638
Failed to add 1639
Failed to add 1640
Failed to add 1641
Failed to add 1642
Failed to add 1643
Failed to add 1644
Failed to add 1645
Failed to add 1646
Failed to add 1647
Failed to add 1648
Failed to add 1649
Failed to add 1650
Failed to add 1651
Failed to add 1652
Failed to add 1653
Failed to add 1654
Failed to add 1655
Failed to add 1656
Failed to add 1657
Failed to add 1658
Failed to add 1659
Failed to add 1660
Failed to add 1661
Failed to add 1662
Failed to add 1663
Failed to add 1664
Failed to add 1665
Failed to add 1666
Failed to add 1667
Failed to add 1668
Failed to add 1669
Failed to add 1670
Failed to add 1671
Failed to ad

In [140]:
prod_df = pd.concat(frames, ignore_index=True)

In [143]:
twenty = .2 * len(prod_df)
test_start, test_end = len(prod_df) - twenty, len(prod_df)
print(test_start, test_end)

94696.0 118370


In [145]:
dep_var = "resource_template"
cat_names = ["Subject", "Predicate", "Object"]
procs =[Categorify]
test = TabularList(prod_df.iloc[94696:118370].copy(),
                   cat_names=cat_names)

In [148]:
data = (TabularList.from_df(prod_df,
                            cat_names=cat_names,
                            procs=procs)
        .split_by_idx(list(range(94696,118370)))
        .label_from_df(cols="resource_template")
        .add_test(test)
        .databunch())

In [149]:
data.show_batch(rows=10)

Subject,Predicate,Object,target
http://id.loc.gov/vocabulary/menclvl/f,http://www.w3.org/2000/01/rdf-schema#label,Full,Yale:RT:BF2:Monograph:Work:CtY-BR
https://Nd85931ca673d42eabe69384314d2887f.sinopia.io/.well-known/genid/rdflib/b35,http://id.loc.gov/ontologies/bibframe/identifiedBy,https://Nd85931ca673d42eabe69384314d2887f.sinopia.io/.well-known/genid/rdflib/b47,Yale:RT:BF2:Monograph:SuperWork:CtY
https://Ndd57c5e691da467398a5e94d2c8e8d25.sinopia.io/.well-known/genid/rdflib/b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/ns/prov#Activity,Yale:RT:BF2:Monograph:Work:CtY-BR
https://Ne04bf098848140e6b1f08a7d675423e8.sinopia.io/.well-known/genid/rdflib/b3,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://id.loc.gov/ontologies/bibframe/ShelfMark,Yale:RT:BF2:Monograph:Item-CtY-BR
https://Nf869f4d84ece4fbaa57570061192d04a.sinopia.io/.well-known/genid/rdflib/b10,http://www.w3.org/ns/prov#atTime,2020-03-03T15:04:35.614000+00:00,Yale:RT:BF2:Monograph:Instance:CtY
https://trellis.sinopia.io/repository/alberta/860842a1-8129-4b6e-911a-717638e7af81,http://id.loc.gov/ontologies/bibframe/subject,http://id.loc.gov/authorities/subjects/sh85076638,ld4p:RT:bf2:Monograph:Work:Un-nested
https://trellis.sinopia.io/repository/cornell/9340f86e-4353-43d4-8e5f-3d33474dab7a,http://id.loc.gov/ontologies/bibframe/issuance,http://id.loc.gov/vocabulary/issuance/mono,CornellSinatra45:Instance
https://trellis.sinopia.io/repository/yale/71d47269-e8b8-4148-a80e-70231888a759,http://id.loc.gov/ontologies/bibframe/title,https://N1ebe2bfbedfe4c35a732a3a46b98f5fa.sinopia.io/.well-known/genid/rdflib/b2,Yale:RT:BF2:Monograph:Work:CtY
https://N7e7ae784d76c4f7ba423e89196eecb54.sinopia.io/.well-known/genid/rdflib/b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://id.loc.gov/ontologies/bibframe/Audio,CornellSinatra45:Work
https://N2c8b118c04f84bdb83fecef8385271a0.sinopia.io/.well-known/genid/rdflib/b11,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://id.loc.gov/ontologies/bibframe/Person,Yale:RT:BF2:Monograph:SuperWork:CtY-BR


In [151]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [153]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.489271,5.085945,0.139110,06:04


In [154]:
row = prod_df.iloc[5]

In [161]:
pred = learn.predict(row)

In [162]:
pred

(Category tensor(43),
 tensor(43),
 tensor([1.6406e-05, 3.6635e-05, 3.1323e-07, 1.7295e-07, 5.5846e-08, 1.4659e-06,
         2.8425e-08, 1.6327e-07, 9.1508e-06, 1.6776e-08, 5.5848e-03, 8.4380e-08,
         2.3532e-06, 1.9649e-06, 6.5554e-05, 3.8854e-07, 8.2281e-10, 7.0165e-08,
         5.3348e-08, 6.3051e-06, 9.7275e-06, 8.7225e-08, 3.8114e-06, 3.1672e-08,
         1.2863e-06, 1.1940e-09, 8.1704e-09, 1.7951e-10, 5.3273e-06, 2.8888e-06,
         5.5764e-08, 3.4818e-06, 8.3740e-09, 4.3611e-08, 1.8566e-06, 7.5834e-05,
         4.9946e-05, 6.8442e-04, 1.1200e-07, 4.1586e-06, 3.5722e-05, 2.6788e-06,
         2.2305e-08, 9.9336e-01, 5.0871e-06, 1.2145e-08, 6.5087e-07, 4.4035e-06,
         3.1881e-06, 2.1333e-09, 1.0485e-06, 3.4327e-08, 2.0945e-07, 5.0255e-08,
         4.1720e-08, 6.5140e-07, 1.6551e-05, 1.2730e-07, 5.3615e-08, 1.9723e-08,
         9.9490e-09]))

In [169]:
pred[0]
learn.data.classes[43]

'ld4p:RT:bf2:Monograph:Instance'

In [181]:
len(prod_df)

118370

In [182]:
for r in range(50000,50036):
    triple = prod_df.iloc[r]
    pred = learn.predict(triple)
    print(f"{r} {triple['resource_template']} predication: {learn.data.classes[int(pred[1])]}")

50000 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50001 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50002 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50003 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50004 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50005 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Instance:CtY
50006 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50007 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50008 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50009 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Instance:CtY
50010 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Instance:CtY
50011 Yale:RT:BF2:Monograph:Item-CtY predication: Yale:RT:BF2:Monograph:Item-CtY
50012 Yale:RT:BF

In [175]:
prd2 = learn.predict(prod_df.iloc[4000])
prd2[1]

tensor(0)

In [176]:
learn.data.classes[0]

'CornellSinatra45:Instance'

In [91]:
old_value = series[1]
old_value = old_value.replace("<http", "http")
old_value = old_value.replace("\&gt;", "")
print(old_value)

http://www.w3.org/2000/01/rdf-schema#label>


In [89]:
series.pop(3)

'_:N88b0a1d5d1c646d58cc99ba9b09e5e0b'

In [90]:
series

0       _:N88b0a1d5d1c646d58cc99ba9b09e5e0b_:b17
1    http://www.w3.org/2000/01/rdf-schema#label>
2                                      "Alma"@en
4                                              .
dtype: object

In [59]:
b1.skolemize(authority='http://happy.io', base)

rdflib.term.URIRef('http://rdlib.net/.well-known/genid/rdflib/abcde')

In [53]:
df0['Subject'][1]

'b0'

In [12]:
result = prod_graphs[0].query(SUBJ_RT)

In [48]:
df1 = graph_triples_df(prod_graphs[1])

In [49]:
df1['Subject'][1]

'https://trellis.sinopia.io/.well-known/genid/rdflib/b17'

In [30]:
df0.append(df1)

,Subject,Predicate,Object,resource_template
0,b17,http://www.w3.org/2000/01/rdf-schema#label,Alma,ld4p:RT:bf2:Monograph:Instance
1,b0,http://id.loc.gov/ontologies/bibframe/temporal...,960-1644,ld4p:RT:bf2:Monograph:Instance
2,b0,http://id.loc.gov/ontologies/bibframe/classifi...,b13,ld4p:RT:bf2:Monograph:Instance
3,b0,http://id.loc.gov/ontologies/bibframe/notation,b9,ld4p:RT:bf2:Monograph:Instance
4,b4,http://id.loc.gov/ontologies/bibframe/date,2017,ld4p:RT:bf2:Monograph:Instance
...,...,...,...,...
80,https://trellis.sinopia.io/repository/ucdavis/...,http://id.loc.gov/ontologies/bibframe/responsi...,柳書琴主編,ld4p:RT:bf2:Monograph:Instance
81,b7,http://id.loc.gov/ontologies/bflc/encodingLevel,http://id.loc.gov/vocabulary/menclvl/f,ld4p:RT:bf2:Monograph:Instance
82,https://trellis.sinopia.io/repository/ucdavis/...,http://www.w3.org/ns/prov#wasGeneratedBy,b12,ld4p:RT:bf2:Monograph:Instance
83,b2,http://id.loc.gov/ontologies/bibframe/agent,b15,ld4p:RT:bf2:Monograph:Instance


In [29]:
d_all.shape

(92, 4)

In [19]:
result.bindings[0]['resource_template']

rdflib.term.Literal('ld4p:RT:bf2:Monograph:Instance')